### ReAct Agent Architecture 


Aim 

This is the intuition behind ReAct, a general agent architecture.

1. <b>act</b> - let the model call specific tools 
2. <b>observe</b> - pass the tool output back to the model.
3. <b>reason</b> - let the model reason about the tool output to decide what to do next(eg. call another tool or just respond directly.)

In [ ]:
impo